Muhammad Qasim 22I-1994 | Ayaan Khan 22I-2066 |
Ahmed Luqman 22I-2018

## Import Required Libraries

In [12]:
import copy
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from collections import defaultdict, namedtuple
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms

from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier

# Try to import TPU support
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    TPU_AVAILABLE = True
    print("TPU support detected!")
except ImportError:
    TPU_AVAILABLE = False
    print("TPU support not available, will use CPU/CUDA")

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Create directory for saving models
os.makedirs('saved_models', exist_ok=True)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"TPU available: {TPU_AVAILABLE}")
print(f"Models will be saved to: ./saved_models/")

TPU support not available, will use CPU/CUDA
Libraries imported successfully!
PyTorch version: 2.8.0
CUDA available: False
TPU available: False
Models will be saved to: ./saved_models/


## Model Definitions

In [13]:
class SimpleCNN(nn.Module):
    """Simple CNN for MNIST and Fashion-MNIST"""
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7 if input_channels==1 else 64*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        return self.fc(self.conv(x))

def get_resnet18(num_classes=10):
    """ResNet18 adapted for CIFAR-10"""
    model = torchvision.models.resnet18(pretrained=False)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

print("Model architectures defined!")

Model architectures defined!


## Utility Functions

In [14]:
def model_to_vector(model):
    """Convert model parameters to a flat numpy vector"""
    return torch.cat([p.detach().flatten() for p in model.parameters()]).cpu().numpy()

def vector_to_model(model, vec):
    """Load vector into model parameters"""
    ptr = 0
    for p in model.parameters():
        num = p.numel()
        p.data.copy_(torch.tensor(vec[ptr:ptr+num]).view_as(p.data))
        ptr += num

def subtract_model_params(m_new, m_old):
    """Compute parameter update: new - old"""
    v_new = model_to_vector(m_new)
    v_old = model_to_vector(m_old)
    return v_new - v_old

def set_seed(seed):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

print("Utility functions defined!")

Utility functions defined!


## Attack Implementations

In [15]:
def sign_flip_update(grad_vec):
    """Sign flipping attack: negate the gradient update"""
    return -grad_vec

def lie_attack_simulation(global_mean, global_std, beta=1.5):
    """LIE attack: craft malicious update near mean but in negative direction"""
    sign = -1.0
    return global_mean + sign * beta * global_std

print("Attack functions defined!")

Attack functions defined!


## Federated Learning Utilities

In [16]:
def local_train(model, train_loader, device, epochs=1, lr=0.01):
    """Train model locally for specified epochs"""
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
        
    for e in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

def fedavg(models):
    """Federated Averaging: average model parameters"""
    avg = {}
    for k in models[0].keys():
        avg[k] = sum([m[k].float() for m in models]) / len(models)
    return avg

def evaluate(model, dataloader, device):
    """Evaluate model accuracy on test set"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    
    return correct / total

print("Federated learning functions defined!")

Federated learning functions defined!


## NC-FLD Defense: Weight Pruning, PCA, and Classification

In [17]:
def select_top_by_magnitude(grad_vec, keep_ratio):
    """Select top parameters by magnitude (weight pruning step)"""
    magnitudes = np.abs(grad_vec)
    threshold = np.quantile(magnitudes, 1 - keep_ratio)
    mask = magnitudes > threshold
    return grad_vec[mask], mask

def build_feature_matrix(selected_vectors):
    """Build feature matrix from selected parameters (pad to max length)"""
    max_len = max([v.shape[0] for v in selected_vectors])
    X = np.zeros((len(selected_vectors), max_len), dtype=np.float32)
    for i, v in enumerate(selected_vectors):
        X[i, :v.shape[0]] = v
    return X

def apply_pca(X, n_components=2):
    """Apply PCA for dimensionality reduction"""
    pca = PCA(n_components=n_components)
    Z = pca.fit_transform(X)
    return Z, pca

def classify_embeddings(Z, method='ocsvm'):
    """Classify clients as benign (True) or malicious (False)"""
    if method == 'ocsvm':
        clf = OneClassSVM(kernel='rbf', nu=0.1, gamma='scale')
        clf.fit(Z)
        preds = clf.predict(Z)  # +1 inlier, -1 outlier
        return (preds == 1).astype(bool)
    
    elif method == 'agg':
        cl = AgglomerativeClustering(n_clusters=2, linkage='average')
        labels = cl.fit_predict(Z)
        counts = np.bincount(labels)
        benign_label = np.argmax(counts)
        return (labels == benign_label).astype(bool)
    
    elif method == 'knn':
        k = 4
        kn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
        agg = AgglomerativeClustering(n_clusters=2)
        pseudo = agg.fit_predict(Z)
        counts = np.bincount(pseudo)
        benign_label = np.argmax(counts)
        y = (pseudo == benign_label).astype(int)
        kn.fit(Z, y)
        preds = kn.predict(Z)
        return (preds == 1).astype(bool)
    
    else:
        raise ValueError(f'Unknown defense method: {method}')

print("NC-FLD defense functions defined!")

NC-FLD defense functions defined!


## Main Simulation Function

In [18]:
# Create named tuple for experiment configuration
ExpConfig = namedtuple('ExpConfig', ['dataset', 'clients', 'rounds', 'local_epochs', 
                                     'lr', 'attack', 'malicious_ratio', 'defense', 
                                     'keep_ratio', 'seed', 'log_interval'])

def simulate(config):
    """
    Run federated learning simulation with NC-FLD defense
    
    Args:
        config: ExpConfig object with experiment parameters
    
    Returns:
        results: List of test accuracies per round
    """
    # Device setup - TPU first, then CUDA, then CPU
    try:
        if TPU_AVAILABLE:
            device = xm.xla_device()
            print(f"Using TPU device: {device}")
        elif torch.cuda.is_available():
            device = torch.device('cuda')
            print("Using CUDA device")
        else:
            device = torch.device('cpu')
            print("Using CPU device")
    except:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
    
    set_seed(config.seed)
    
    # Create directory for this experiment's models
    exp_name = f"{config.dataset}_{config.attack}_ratio{config.malicious_ratio}_{config.defense}"
    model_dir = os.path.join('saved_models', exp_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # Dataset setup
    if config.dataset == 'mnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                             download=True, transform=transform)
        testset = torchvision.datasets.MNIST(root='./data', train=False, 
                                            download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 32
        flip_map = {7: 1}
    
    elif config.dataset == 'fmnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, 
                                                     download=True, transform=transform)
        testset = torchvision.datasets.FashionMNIST(root='./data', train=False, 
                                                    download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 25
        flip_map = {0: 6, 6: 0}
    
    elif config.dataset == 'cifar10':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5, 0.5, 0.5), 
                                                          (0.5, 0.5, 0.5))])
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                               download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                              download=True, transform=transform)
        model_fn = lambda: get_resnet18(num_classes=10)
        batch_size = 100
        flip_map = {5: 3}  # dog -> cat
    
    else:
        raise NotImplementedError(f'Dataset {config.dataset} not implemented')
    
    # Create client data shards
    total_train = len(trainset)
    indices = list(range(total_train))
    random.shuffle(indices)
    shards = np.array_split(indices, config.clients)
    
    client_loaders = []
    for s in shards:
        sub = Subset(trainset, s)
        loader = DataLoader(sub, batch_size=batch_size, shuffle=True)
        client_loaders.append(loader)
    
    testloader = DataLoader(testset, batch_size=256, shuffle=False)
    
    # Select malicious clients
    num_mal = int(config.malicious_ratio * config.clients)
    mal_indices = set(random.sample(range(config.clients), num_mal))
    print(f"Malicious clients: {sorted(list(mal_indices))} (count={num_mal})")
    
    # Initialize global model
    global_model = model_fn().to(device)
    global_state = copy.deepcopy(global_model.state_dict())
    
    # Main federated learning rounds
    results = []
    for r in trange(config.rounds, desc='Training Rounds'):
        local_states = []
        grad_vectors = []
        raw_masks = []
        
        # Local training for each client
        for c in range(config.clients):
            model_c = model_fn().to(device)
            model_c.load_state_dict(global_state)
            old_model = copy.deepcopy(model_c)
            
            # Label flip attack: poison training data
            if config.attack == 'label_flip' and c in mal_indices:
                ds_indices = shards[c]
                poison_list = []
                for idx in ds_indices:
                    x, y = trainset[idx]
                    y_ = flip_map.get(int(y), int(y))
                    poison_list.append((x, y_))
                poisoned_loader = DataLoader(poison_list, batch_size=batch_size, shuffle=True)
                local_train(model_c, poisoned_loader, device, 
                          epochs=config.local_epochs, lr=config.lr)
            else:
                local_train(model_c, client_loaders[c], device, 
                          epochs=config.local_epochs, lr=config.lr)
            
            # Compute update vector
            diff_vec = subtract_model_params(model_c, old_model)
            
            # Model poisoning attacks
            if c in mal_indices and config.attack == 'sign_flip':
                diff_vec = sign_flip_update(diff_vec)
            
            if c in mal_indices and config.attack == 'lie':
                if len(grad_vectors) > 0:
                    global_mean = np.mean(np.stack(grad_vectors), axis=0)
                    global_std = np.std(np.stack(grad_vectors), axis=0) + 1e-6
                    diff_vec = lie_attack_simulation(global_mean, global_std, beta=1.0)
                else:
                    diff_vec = sign_flip_update(diff_vec)
            
            grad_vectors.append(diff_vec)
            local_states.append(model_c.state_dict())
        
        # Synchronize if using TPU
        if TPU_AVAILABLE:
            xm.mark_step()
        
        # NC-FLD Defense: Weight Pruning
        selected_vecs = []
        for vec in grad_vectors:
            sel, mask = select_top_by_magnitude(vec, config.keep_ratio)
            selected_vecs.append(sel)
            raw_masks.append(mask)
        
        # Build feature matrix and apply PCA
        X = build_feature_matrix(selected_vecs)
        Z, pca = apply_pca(X, n_components=2)
        
        # Classify clients
        benign_mask = classify_embeddings(Z, method=config.defense)
        benign_indices = [i for i, b in enumerate(benign_mask) if b]
        
        if len(benign_indices) == 0:
            print("Warning: all clients classified malicious; skipping aggregation")
            continue
        
        # Aggregate benign clients (FedAvg)
        benign_states = [local_states[i] for i in benign_indices]
        new_global_state = fedavg(benign_states)
        global_state = new_global_state
        
        # Evaluate
        global_model.load_state_dict(global_state)
        acc = evaluate(global_model, testloader, device)
        results.append(acc)
        
        if (r + 1) % config.log_interval == 0:
            print(f"Round {r+1}/{config.rounds} - Test Acc: {acc:.4f} - "
                  f"Included: {len(benign_indices)}/{config.clients}")
    
    final_acc = results[-1] if len(results) > 0 else None
    print(f"Final test accuracy: {final_acc:.4f}")
    
    # Save only the final model after all rounds complete
    # Move model to CPU before saving to ensure compatibility
    cpu_global_state = {k: v.cpu() for k, v in global_state.items()}
    
    torch.save({
        'round': config.rounds,
        'model_state_dict': cpu_global_state,
        'config': config._asdict(),
        'accuracy': final_acc,
        'all_round_accuracies': results,
        'malicious_clients': list(mal_indices)
    }, os.path.join(model_dir, f'model_final_round_{config.rounds}.pt'))
    
    print(f"Final model saved to: {model_dir}/model_final_round_{config.rounds}.pt")
    return results, final_acc

print("Simulation function defined!")

Simulation function defined!


## Model Loading Utility

In [19]:
def load_saved_model(model_path, dataset='fmnist'):
    """
    Load a saved model checkpoint
    
    Args:
        model_path: Path to the saved model file
        dataset: Dataset name to initialize correct model architecture
    
    Returns:
        model: Loaded model
        checkpoint: Full checkpoint dictionary with metadata
    """
    # Initialize model architecture
    if dataset in ['mnist', 'fmnist']:
        model = SimpleCNN(input_channels=1, num_classes=10)
    elif dataset == 'cifar10':
        model = get_resnet18(num_classes=10)
    else:
        raise ValueError(f"Unknown dataset: {dataset}")
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Loaded model from round {checkpoint['round']}")
    if checkpoint['accuracy'] is not None:
        print(f"Accuracy: {checkpoint['accuracy']:.4f}")
    
    return model, checkpoint

# Example usage:
# model, checkpoint = load_saved_model('saved_models/fmnist_label_flip_ratio0.1_ocsvm/model_round_100.pt', dataset='fmnist')

print("Model loading utility defined!")

Model loading utility defined!


## Initialize Results Storage

In [20]:
# Storage for all experiment results
all_results = []

print("Results storage initialized!")

Results storage initialized!


---
# Fashion-MNIST Experiments (100 clients)
## All Attacks

In [ ]:
# Fashion-MNIST - All Attacks, Ratios, and Defenses
for attack in ['label_flip', 'sign_flip', 'lie']:
    for ratio in [0.1, 0.2, 0.3, 0.4]:
        for defense in ['ocsvm', 'agg', 'knn']:
            config = ExpConfig(dataset='fmnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                               attack=attack, malicious_ratio=ratio, defense=defense,
                               keep_ratio=0.15, seed=42, log_interval=10)
            round_accs, final_acc = simulate(config)
            all_results.append({'dataset': 'fmnist', 'attack': attack, 'malicious_ratio': ratio,
                               'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [5, 8, 20, 41, 54, 59, 72, 80, 83, 84] (count=10)


Training Rounds:   0%|          | 0/100 [00:05<?, ?it/s]



KeyboardInterrupt: 

## Check Existing Trained Models

In [21]:
def check_model_exists(dataset, attack, ratio, defense, rounds=100):
    """Check if a trained model already exists"""
    exp_name = f"{dataset}_{attack}_ratio{ratio}_{defense}"
    model_path = os.path.join('saved_models', exp_name, f'model_final_round_{rounds}.pt')
    return os.path.exists(model_path), model_path

def get_trained_models_info():
    """Get information about all trained models"""
    trained_models = []
    
    if not os.path.exists('saved_models'):
        return trained_models
    
    for exp_dir in os.listdir('saved_models'):
        exp_path = os.path.join('saved_models', exp_dir)
        if os.path.isdir(exp_path):
            # Look for final model files
            for file in os.listdir(exp_path):
                if file.startswith('model_final_round_') and file.endswith('.pt'):
                    model_path = os.path.join(exp_path, file)
                    try:
                        checkpoint = torch.load(model_path, map_location='cpu')
                        trained_models.append({
                            'experiment': exp_dir,
                            'round': checkpoint['round'],
                            'accuracy': checkpoint.get('accuracy', 'N/A'),
                            'path': model_path,
                            'config': checkpoint.get('config', {})
                        })
                    except:
                        pass
    
    return trained_models

# Check what models are already trained
print("="*80)
print("CHECKING EXISTING TRAINED MODELS")
print("="*80)

trained_models = get_trained_models_info()

if len(trained_models) == 0:
    print("\nNo trained models found. All experiments will be run.")
else:
    print(f"\nFound {len(trained_models)} trained models:")
    print("-"*80)
    for i, model in enumerate(trained_models, 1):
        print(f"{i}. {model['experiment']}")
        acc = model['accuracy']
        if isinstance(acc, float):
            print(f"   Round: {model['round']}, Accuracy: {acc:.4f}")
        else:
            print(f"   Round: {model['round']}, Accuracy: {acc}")
        print(f"   Path: {model['path']}")
        print()

print("="*80)

CHECKING EXISTING TRAINED MODELS

Found 32 trained models:
--------------------------------------------------------------------------------
1. fmnist_lie_ratio0.3_agg
   Round: 100, Accuracy: 0.8323
   Path: saved_models/fmnist_lie_ratio0.3_agg/model_final_round_100.pt

2. fmnist_lie_ratio0.1_knn
   Round: 100, Accuracy: 0.8324
   Path: saved_models/fmnist_lie_ratio0.1_knn/model_final_round_100.pt

3. fmnist_lie_ratio0.2_agg
   Round: 100, Accuracy: 0.8321
   Path: saved_models/fmnist_lie_ratio0.2_agg/model_final_round_100.pt

4. fmnist_lie_ratio0.2_knn
   Round: 100, Accuracy: 0.8321
   Path: saved_models/fmnist_lie_ratio0.2_knn/model_final_round_100.pt

5. fmnist_lie_ratio0.1_agg
   Round: 100, Accuracy: 0.8323
   Path: saved_models/fmnist_lie_ratio0.1_agg/model_final_round_100.pt

6. fmnist_sign_flip_ratio0.4_agg
   Round: 100, Accuracy: 0.8324
   Path: saved_models/fmnist_sign_flip_ratio0.4_agg/model_final_round_100.pt

7. fmnist_sign_flip_ratio0.4_knn
   Round: 100, Accuracy: 0.83

## Smart Training: Skip Already Trained Models

In [22]:
# Fashion-MNIST - All Attacks, Ratios, and Defenses (Skip if already trained)
print("="*80)
print("FASHION-MNIST EXPERIMENTS - SMART TRAINING")
print("="*80)

total_experiments = 0
skipped_experiments = 0
trained_experiments = 0

for attack in ['label_flip', 'sign_flip', 'lie']:
    for ratio in [0.1, 0.2, 0.3, 0.4]:
        for defense in ['ocsvm', 'agg', 'knn']:
            total_experiments += 1
            
            # Check if model already exists
            exists, model_path = check_model_exists('fmnist', attack, ratio, defense, rounds=100)
            
            if exists:
                print(f"\n✓ SKIPPING: fmnist_{attack}_ratio{ratio}_{defense} (already trained)")
                skipped_experiments += 1
                
                # Load existing results
                try:
                    checkpoint = torch.load(model_path, map_location='cpu')
                    final_acc = checkpoint.get('accuracy')
                    round_accs = checkpoint.get('all_round_accuracies', [])
                    
                    all_results.append({
                        'dataset': 'fmnist', 
                        'attack': attack, 
                        'malicious_ratio': ratio,
                        'defense': defense, 
                        'final_accuracy': final_acc, 
                        'round_accuracies': round_accs
                    })
                    print(f"  Loaded: Accuracy = {final_acc:.4f}")
                except Exception as e:
                    print(f"  Warning: Could not load checkpoint - {e}")
            else:
                print(f"\n→ TRAINING: fmnist_{attack}_ratio{ratio}_{defense}")
                trained_experiments += 1
                
                config = ExpConfig(
                    dataset='fmnist', 
                    clients=100, 
                    rounds=100, 
                    local_epochs=1, 
                    lr=0.01,
                    attack=attack, 
                    malicious_ratio=ratio, 
                    defense=defense,
                    keep_ratio=0.15, 
                    seed=42, 
                    log_interval=10
                )
                
                round_accs, final_acc = simulate(config)
                
                all_results.append({
                    'dataset': 'fmnist', 
                    'attack': attack, 
                    'malicious_ratio': ratio,
                    'defense': defense, 
                    'final_accuracy': final_acc, 
                    'round_accuracies': round_accs
                })

print("\n" + "="*80)
print("FASHION-MNIST EXPERIMENTS SUMMARY")
print("="*80)
print(f"Total experiments: {total_experiments}")
print(f"Skipped (already trained): {skipped_experiments}")
print(f"Newly trained: {trained_experiments}")
print("="*80)

FASHION-MNIST EXPERIMENTS - SMART TRAINING

✓ SKIPPING: fmnist_label_flip_ratio0.1_ocsvm (already trained)
  Loaded: Accuracy = 0.8315

✓ SKIPPING: fmnist_label_flip_ratio0.1_agg (already trained)
  Loaded: Accuracy = 0.8316

✓ SKIPPING: fmnist_label_flip_ratio0.1_knn (already trained)
  Loaded: Accuracy = 0.8321

✓ SKIPPING: fmnist_label_flip_ratio0.2_ocsvm (already trained)
  Loaded: Accuracy = 0.8306

✓ SKIPPING: fmnist_label_flip_ratio0.2_agg (already trained)
  Loaded: Accuracy = 0.8310

✓ SKIPPING: fmnist_label_flip_ratio0.2_knn (already trained)
  Loaded: Accuracy = 0.8319

✓ SKIPPING: fmnist_label_flip_ratio0.3_ocsvm (already trained)
  Loaded: Accuracy = 0.8256

✓ SKIPPING: fmnist_label_flip_ratio0.3_agg (already trained)
  Loaded: Accuracy = 0.8316

✓ SKIPPING: fmnist_label_flip_ratio0.3_knn (already trained)
  Loaded: Accuracy = 0.8321

✓ SKIPPING: fmnist_label_flip_ratio0.4_ocsvm (already trained)
  Loaded: Accuracy = 0.8091

✓ SKIPPING: fmnist_label_flip_ratio0.4_agg (alre

Training Rounds:  10%|█         | 10/100 [03:00<26:37, 17.75s/it]

Round 10/100 - Test Acc: 0.7233 - Included: 70/100


Training Rounds:  20%|██        | 20/100 [06:02<24:25, 18.32s/it]

Round 20/100 - Test Acc: 0.7575 - Included: 70/100


...

Round 80/100 - Test Acc: 0.8196 - Included: 60/100


Training Rounds:  90%|█████████ | 90/100 [34:25<03:46, 22.61s/it]

Round 90/100 - Test Acc: 0.8278 - Included: 60/100


Training Rounds: 100%|██████████| 100/100 [38:14<00:00, 22.94s/it]

Round 100/100 - Test Acc: 0.8324 - Included: 60/100
Final test accuracy: 0.8324
Final model saved to: saved_models/fmnist_lie_ratio0.4_knn/model_final_round_100.pt

FASHION-MNIST EXPERIMENTS SUMMARY
Total experiments: 36
Skipped (already trained): 32
Newly trained: 4


## Summary Tables

In [23]:
# Create pivot table for easy viewing
summary_pivot = df.pivot_table(
    values='final_accuracy',
    index=['dataset', 'attack', 'malicious_ratio'],
    columns='defense',
    aggfunc='mean'
)

print("\n" + "="*80)
print("SUMMARY: Final Test Accuracy by Configuration")
print("="*80)
print(summary_pivot.to_string())

summary_pivot.to_csv('summary_accuracy_table.csv')
summary_pivot

NameError: name 'df' is not defined

In [ ]:
# Individual dataset tables
for dataset in df['dataset'].unique():
    print(f"\n{'='*80}")
    print(f"{dataset.upper()} - Detailed Results")
    print(f"{'='*80}")
    
    df_dataset = df[df['dataset'] == dataset]
    pivot = df_dataset.pivot_table(
        values='final_accuracy',
        index=['attack', 'malicious_ratio'],
        columns='defense',
        aggfunc='mean'
    )
    print(pivot.to_string())
    pivot.to_csv(f'{dataset}_results_table.csv')
    display(pivot)

## Visualization 1: Accuracy vs Malicious Ratio

In [ ]:
# Plot accuracy vs malicious ratio
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('NC-FLD Defense Performance: Accuracy vs Malicious Ratio', 
             fontsize=16, fontweight='bold', y=0.995)

datasets = df['dataset'].unique()
attacks = df['attack'].unique()

for i, dataset in enumerate(datasets):
    for j, attack in enumerate(attacks):
        ax = axes[i, j]
        data = df[(df['dataset'] == dataset) & (df['attack'] == attack)]
        
        if len(data) > 0:
            for defense in ['ocsvm', 'agg', 'knn']:
                defense_data = data[data['defense'] == defense].sort_values('malicious_ratio')
                if len(defense_data) > 0:
                    ax.plot(defense_data['malicious_ratio'], 
                           defense_data['final_accuracy'],
                           marker='o', label=defense.upper(), linewidth=2.5, markersize=8)
            
            ax.set_xlabel('Malicious Client Ratio', fontsize=11)
            ax.set_ylabel('Test Accuracy', fontsize=11)
            ax.set_title(f'{dataset.upper()} - {attack.replace("_", " ").title()}', 
                        fontsize=12, fontweight='bold')
            ax.legend(fontsize=10)
            ax.grid(True, alpha=0.3)
            ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('accuracy_vs_malicious_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

## Visualization 2: Defense Method Comparison

In [ ]:
# Bar plot comparing defense methods
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Average Defense Performance by Dataset', fontsize=16, fontweight='bold')

for i, dataset in enumerate(datasets):
    ax = axes[i]
    data = df[df['dataset'] == dataset]
    defense_avg = data.groupby('defense')['final_accuracy'].mean().sort_values(ascending=False)
    
    colors = ['#2ecc71', '#3498db', '#e74c3c']
    bars = ax.bar(range(len(defense_avg)), defense_avg.values, color=colors, alpha=0.8, edgecolor='black')
    ax.set_xticks(range(len(defense_avg)))
    ax.set_xticklabels([d.upper() for d in defense_avg.index], fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Test Accuracy', fontsize=12)
    ax.set_title(f'{dataset.upper()}', fontsize=13, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
               f'{height:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('defense_method_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Visualization 3: Attack Impact Heatmap

In [ ]:
# Heatmap showing attack impact
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Attack Impact on Accuracy (Averaged Across Defenses)', 
             fontsize=16, fontweight='bold')

for i, dataset in enumerate(datasets):
    ax = axes[i]
    data = df[df['dataset'] == dataset]
    
    heatmap_data = data.pivot_table(
        values='final_accuracy',
        index='attack',
        columns='malicious_ratio',
        aggfunc='mean'
    )
    
    sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='RdYlGn', 
                vmin=0, vmax=1, ax=ax, cbar_kws={'label': 'Accuracy'},
                linewidths=0.5, linecolor='gray')
    ax.set_title(f'{dataset.upper()}', fontsize=13, fontweight='bold')
    ax.set_xlabel('Malicious Ratio', fontsize=11)
    ax.set_ylabel('Attack Type', fontsize=11)
    ax.set_yticklabels([a.replace('_', ' ').title() for a in heatmap_data.index], rotation=0)

plt.tight_layout()
plt.savefig('attack_impact_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Statistical Analysis

In [ ]:
print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)

print("\n1. Overall Accuracy Statistics:")
print(df['final_accuracy'].describe())

print("\n2. Best Performing Configurations (Top 10):")
top_10 = df.nlargest(10, 'final_accuracy')[['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy']]
print(top_10.to_string(index=False))

print("\n3. Worst Performing Configurations (Bottom 10):")
bottom_10 = df.nsmallest(10, 'final_accuracy')[['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy']]
print(bottom_10.to_string(index=False))

print("\n4. Average Accuracy by Defense Method:")
defense_stats = df.groupby('defense')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(defense_stats.to_string())

print("\n5. Average Accuracy by Attack Type:")
attack_stats = df.groupby('attack')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(attack_stats.to_string())

print("\n6. Average Accuracy by Dataset:")
dataset_stats = df.groupby('dataset')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(dataset_stats.to_string())

print("\n7. Average Accuracy by Malicious Ratio:")
ratio_stats = df.groupby('malicious_ratio')['final_accuracy'].agg(['mean', 'std', 'min', 'max'])
print(ratio_stats.to_string())

## Generate Comprehensive Report

In [ ]:
# Create comprehensive text report
with open('nc_fld_experiment_report.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("NC-FLD FEDERATED LEARNING DEFENSE\n")
    f.write("COMPREHENSIVE EXPERIMENT REPORT\n")
    f.write("="*80 + "\n\n")
    
    f.write(f"Total Experiments Completed: {len(df)}\n\n")
    
    f.write("="*80 + "\n")
    f.write("OVERALL SUMMARY TABLE\n")
    f.write("="*80 + "\n\n")
    f.write(summary_pivot.to_string())
    f.write("\n\n")
    
    for dataset in datasets:
        f.write("="*80 + "\n")
        f.write(f"{dataset.upper()} DETAILED RESULTS\n")
        f.write("="*80 + "\n\n")
        df_dataset = df[df['dataset'] == dataset]
        pivot = df_dataset.pivot_table(
            values='final_accuracy',
            index=['attack', 'malicious_ratio'],
            columns='defense',
            aggfunc='mean'
        )
        f.write(pivot.to_string())
        f.write("\n\n")
    
    f.write("="*80 + "\n")
    f.write("KEY FINDINGS\n")
    f.write("="*80 + "\n\n")
    
    best_defense = df.groupby('defense')['final_accuracy'].mean().idxmax()
    best_defense_acc = df.groupby('defense')['final_accuracy'].mean().max()
    f.write(f"Best Defense Method: {best_defense.upper()} (Avg Accuracy: {best_defense_acc:.4f})\n")
    
    most_robust = df.groupby('attack')['final_accuracy'].mean().idxmax()
    f.write(f"Most Robust Against: {most_robust.replace('_', ' ').title()} Attack\n")
    
    best_dataset = df.groupby('dataset')['final_accuracy'].mean().idxmax()
    f.write(f"Best Dataset Performance: {best_dataset.upper()}\n")

print("\n" + "="*80)
print("ALL EXPERIMENTS COMPLETED SUCCESSFULLY!")
print("="*80)
print("\nGenerated Files:")
print("  1. nc_fld_experiment_results.csv - Raw results data")
print("  2. summary_accuracy_table.csv - Summary pivot table")
print("  3. mnist_results_table.csv - MNIST detailed results")
print("  4. fmnist_results_table.csv - Fashion-MNIST detailed results")
print("  5. cifar10_results_table.csv - CIFAR-10 detailed results")
print("  6. accuracy_vs_malicious_ratio.png - Line plots")
print("  7. defense_method_comparison.png - Bar charts")
print("  8. attack_impact_heatmap.png - Heatmaps")
print("  9. nc_fld_experiment_report.txt - Comprehensive text report")
print("\n" + "="*80)